Library

In [2]:
import math, os, sys
import numpy as np
import itertools
import skimage
from skimage import measure, morphology
from skimage.transform import resize
from skimage.io import imsave, imread
from skimage.filters import threshold_otsu

import tensorflow as tf
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


import matplotlib.image as mpimg
import matplotlib.pyplot as plt
plt.style.use('default')
from scipy import ndimage


from PIL import Image
import pandas as pd

read and unzip data

In [3]:
import zipfile

local_zip = './data/data_capstone_project.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('./data')
zip_ref.close()

define each directory 

In [4]:
base_dir = './data/data_capstone_project'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Directory with our training pictures
train_dermatitis_dir = os.path.join(train_dir, 'dermatitis')
train_eczema_dir = os.path.join(train_dir, 'eczema')
train_psoriasis_dir = os.path.join(train_dir, 'psoriasis')

# Directory with our validation pictures
test_dermatitis_dir = os.path.join(test_dir, 'dermatitis')
test_eczema_dir = os.path.join(test_dir, 'eczema')
test_psoriasis_dir = os.path.join(test_dir, 'psoriasis')

In [ ]:
train_dermatitis_fnames = os.listdir( train_dermatitis_dir )
train_eczema_fnames = os.listdir( train_eczema_dir )
train_psoriasis_fnames = os.listdir( train_psoriasis_dir )

print(train_dermatitis_fnames[:10])
print(train_eczema_fnames[:10])
print(train_psoriasis_fnames[:10])

total number of images 

In [ ]:
print('total training dermatitis images :', len(os.listdir(      train_dermatitis_dir ) ))
print('total training eczema images :', len(os.listdir(      train_eczema_dir ) ))
print('total training psoriasis images :', len(os.listdir(      train_psoriasis_dir ) ))

print('total testing dermatitis images :', len(os.listdir( test_dermatitis_dir ) ))
print('total testing eczema images :', len(os.listdir( test_eczema_dir ) ))
print('total testing psoriasis images :', len(os.listdir( test_psoriasis_dir ) ))

data preprocessing 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    target_size=(224, 224))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=128,
                                                         class_mode  = 'categorical',
                                                         target_size = (224, 224))


show image

In [ ]:
image = imread('./data/data_capstone_project/train/eczema/eczema-100.jpg')
plt.imshow(image)
plt.show()

Make label.txt

In [ ]:
print (train_generator.class_indices)

label_txt = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('label.txt', 'w') as f:
    f.write(label_txt)

pre-trained model 

In [10]:
import efficientnet.tfkeras as efn
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the EfficientNetB0 base model
base_model = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [11]:
base_model.trainable = False 
base_model.summary()

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 32) 0           stem_bn[0][0]                    
____________________________________________________________________________________

In [12]:
import tensorflow.keras 
from tensorflow.keras import backend as K 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.utils import to_categorical 


In [13]:
from tensorflow.keras import regularizers
model = Sequential()

# Add the pre-trained base model to the sequential model
model.add(base_model)

# Add your own classification layers
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.01)))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 7, 7, 1280)        4049564   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               327936    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

training

In [ ]:
history = model.fit_generator(train_generator,
                                epochs=200,
                                validation_data=test_generator)

In [ ]:
import matplotlib.pyplot as plt

# Check the available keys in the history object
print(history.history.keys())

# Plot accuracy
if 'accuracy' in history.history:
    plt.plot(history.history["accuracy"], label="training accuracy")
if 'val_accuracy' in history.history:
    plt.plot(history.history["val_accuracy"], label="validation accuracy")

plt.legend()
plt.show()

# Plot loss
if 'loss' in history.history:
    plt.plot(history.history["loss"], label="training loss")
if 'val_loss' in history.history:
    plt.plot(history.history["val_loss"], label="validation loss")

plt.legend()
plt.show()


Save Model

In [ ]:
model.save('model12-efnb0.h5')  # saving model 'model12-efnb0.h5'
tf.saved_model.save(model, 'model12-efnb0.pb')


In [ ]:
# Get the number of trainable variables
num_trainable_variables = len(model.trainable_variables)

print('Number of trainable variables = {}'.format(num_trainable_variables))

using model 

In [ ]:
print(train_generator)

In [ ]:
# Obtain a batch of images and labels from the test generator
image_batch, label_batch = next(test_generator)

# Predict the label for a specific image in the batch
n = 20
input_image = image_batch[n][np.newaxis, ...]
print("The actual label is:", label_batch[n])

prediction = model.predict(input_image)
print("The predicted label is:", prediction[0])

evaluasi model 

In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model

model = load_model('model12-efnb0.h5')

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

# Get the predictions for the test dataset
predictions = model.predict(test_generator)

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels from the test dataset generator
true_labels = test_generator.classes

# Create a confusion matrix
confusion_mat = confusion_matrix(true_labels, predicted_labels)

print('Confusion Matrix:')
print(confusion_mat)
